# Automated project setup

### Setup the connection to WFA SDK

**Requires** the windfarmer python package to be installed in your local python environment so you can call ```wf = sdk.Sdk(windfarmer_installation_folder)```
See the setup instructions in the [SDK intro documentation](https://mysoftware.dnv.com/download/public/renewables/windfarmer/manuals/latest/Automation/SDK/sdkIntro.html)

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import sdk
import os
import pandas as pd
import glob

In [ ]:
windfarmer_installation_folder = r'C:\Program Files\DNV\WindFarmer - Analyst 1.6.4.1'
wf = sdk.Sdk(windfarmer_installation_folder)
print(' > SDK is now up and running!')

### Define file paths and inputs
Here we're using the Hawaii demo data, but you could change these to come from config files or csvs etc.

In [ ]:
lattitude = 20.259495
longitude = -155.866667
epsg = 32605

In [ ]:
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..'))
data_path = os.path.join(root_dir, "DemoData", 'Hawaii')
workbook_path = os.path.join(os.getcwd(), 'Hawaii_autosetup.wwx')

gis_data_path = os.path.join(data_path, "GIS data")
elevation_contours_path = os.path.join(gis_data_path, "Elevation contours - Hawaii.map")
elevation_grid_path = os.path.join(gis_data_path, "SRTM gridded elevations - N20W156.grd") # This could also be generated from the contours, or the other way arround
satellite_image =  os.path.join( gis_data_path, "Satellite imagery.jpg"  )
background_image =  os.path.join( gis_data_path, "Background image - Hawaii.png"  )
turbine_types_folder = os.path.join(data_path, "Turbine types")
timeseries_folder = os.path.join(data_path, "Time series")

In [ ]:
masts_df = pd.DataFrame([
    {"name": "M2", "easting": 199186.0, "northing": 2242285.0, "timeseries_file": "M2 time series.txt", "timeseries_load_settings_file": "M2 time series_LoadSettings.xml" },
    {"name": "M3", "easting": 202139.0, "northing": 2241811.0, "timeseries_file": "M3 time series.txt", "timeseries_load_settings_file": "M3 time series_LoadSettings.xml"} 
    ])
masts_df = masts_df.set_index("name")
masts_df

In [ ]:
windfarms_df = pd.DataFrame([
    {"name": "my wind farm", "is_neighbor": False, "turbine_type_name": "Hawaii 2.0 MW", "initiation_mast": "M2", "shapefile_path": os.path.join(data_path, "Turbines", "My wind farm.shp") },
    {"name": "Kohala", "is_neighbor": True, "turbine_type_name": "Generic Turbine 750kW", "initiation_mast": "M3", "shapefile_path": os.path.join(data_path, "Turbines", "Kohala neighbouring wind farm.shp")} 
    ])
windfarms_df = windfarms_df.set_index("name")
windfarms_df

### Save the workbook

In [ ]:
wf.Toolbox.SaveAs(workbook_path)
print(wf.Toolbox.get_CurrentWorkbookPath())

### Set the site location and co-ordinate system

In [ ]:
latLonSiteLocation = wf.Scripting.Location (longitude, lattitude)
wf.Toolbox.RedefineSiteLocation(latLonSiteLocation)

# Set the workbook co-ordinate system if it hasn't been set already
if (wf.Workbook.Geography.Projection == None):
    print("Setting workbook co-ordinate system")
    # EPSG codes provide a consistent way to define co-ordinate systems and this is what we use in scripting. 
    # Search a reference list like the following to find your co-ordinate system as an ESPG: http://spatialreference.org/ref/epsg/ 
    workbookProjection = wf.Toolbox.GetProjectionFromEpsgCode(epsg)
    wf.Toolbox.RedefineWorkbookProjection(workbookProjection)


### Import Elevation GIS data

In [ ]:
print("Importing elevation data")
# this .map file has no associated projection information so this must be specified
wf.Toolbox.ImportElevationContours(elevation_contours_path, wf.Workbook.Geography.Projection)

wf.Toolbox.ImportElevationGrid(elevation_grid_path)
# this .grd elevation grid file has an associated .prj file, "N20W156.prj", descibing the projection 
# so no projection need be specified, reprojection to the workbook projection is handled automatically on import.
# Alternatively you can create the elevation grid from the contours already loaded in the workbook:
# Toolbox.AddElevationGrid( Workbook.Geography.Contours.FirstOrDefault(), 25 );

In [ ]:
print("Importing background image")
wf.Toolbox.ImportBackgroundImage( satellite_image) # The background is georeferenced with a world file and projection file (.pgw and .prj). A .pgw is needed but you can define the projection from like we did for loading contours
wf.Toolbox.ImportBackgroundImage( background_image ) # alternative full island image, uncomment to load this too

### Configure Measurement sites, loading hub height time series

In [ ]:
# remove existing mast if testing
# wf.Workbook.Climate.MeasurementSites.Remove("M2") 

In [ ]:
for mast_name in masts_df.index:
    mast_row = masts_df.loc[mast_name]
    print(mast_row)
    new_mast = wf.Scripting.Mast(mast_name, wf.Scripting.Location(mast_row["easting"], mast_row["northing"]))
    wf.Workbook.Climate.MeasurementSites.Add(new_mast)
    print([mast for mast in wf.Workbook.Climate.MeasurementSites])
    
    # Import time series data in to the workbook for each of the measurement sites
    time_series_file_path = os.path.join(timeseries_folder, mast_row["timeseries_file"])
    time_series_settings_file_path = os.path.join(timeseries_folder, mast_row["timeseries_load_settings_file"])

    wf.Toolbox.MeasurementCampaign.LoadData(  time_series_file_path, time_series_settings_file_path)

### Create frequency distributions and turbulence intensity distributions from loaded time series

In [ ]:
# shortcut to wind analysis toolbox
wa_toolbox = wf.Toolbox.MeasurementCampaign 

In [ ]:
# The names we're using here to get the time series are coming from the names applied in the loader settings file.
# If you want to automate the process (and not type these in) you should define a standard input file, where you can re-use a standard data loader settings file
# The first time you define loader settings, use the GUI and see https://mysoftware.dnv.com/download/public/renewables/windfarmer/manuals/latest/UserGuide/Wind/Measurements/LoadMeasurementData.html
M2_ws60S_Mean = wa_toolbox.GetSpeedTimeSeries("M2~ws60S~Mean") 
M2_ws60N_Mean = wa_toolbox.GetSpeedTimeSeries("M2~ws60N~Mean")
M2_ws60N_SD = wa_toolbox.GetSpeedTimeSeries("M2~ws60N~StdDev")

M2_wd57_Mean = wa_toolbox.GetDirectionTimeSeries("M2~wd57~Mean")

M3_ws94NW_Mean = wa_toolbox.GetSpeedTimeSeries("M3~ws94NW~Mean")
M3_ws94S_SD = wa_toolbox.GetSpeedTimeSeries("M3~ws94S~StdDev")
M3_wd94_Mean = wa_toolbox.GetDirectionTimeSeries("M3~wd94~Mean")

In [ ]:
print("create frequency distributions")
m2_FD_ws60wd57 = wa_toolbox.CreateFrequencyDistribution(M2_ws60N_Mean, M2_wd57_Mean, True, "m2_FD_ws60wd57")
m3_FD_ws94wd94 = wa_toolbox.CreateFrequencyDistribution(M3_ws94NW_Mean, M2_wd57_Mean, True, "m3_FD_ws94wd94")

print("create turbulence intensity distributions")
M2_60m_TI = wa_toolbox.CreateTurbulenceIntensityDistribution(M2_ws60N_Mean, M2_ws60N_SD, M2_wd57_Mean, "M2_60m_TI")
M3_94m_TI = wa_toolbox.CreateTurbulenceIntensityDistribution(M3_ws94NW_Mean, M3_ws94S_SD, M3_wd94_Mean, "M3_94m_TI")

print("Save frequency distributions and turbulence intensity distributions back to the workbook as inputs to WAsP and energy calculations")
wa_toolbox.SaveDistribution(wf.Workbook.Climate.MeasurementSites["M2"], m2_FD_ws60wd57, m2_FD_ws60wd57.Name, 60, True)
wa_toolbox.SaveDistribution(wf.Workbook.Climate.MeasurementSites["M2"], M2_60m_TI, M2_60m_TI.Name, 60, True)
wa_toolbox.SaveDistribution(wf.Workbook.Climate.MeasurementSites["M3"], m3_FD_ws94wd94, m3_FD_ws94wd94.Name, 94, True)
wa_toolbox.SaveDistribution(wf.Workbook.Climate.MeasurementSites["M3"], M3_94m_TI, M3_94m_TI.Name, 94, True)

### Load turbine types

In [ ]:
print("load all turbine types in the folder in to the workbook")

existing_turbine_type_names = [turbine_type.Name for turbine_type in wf.Workbook.TurbineTypes]

for trbxFile in glob.glob( os.path.join(turbine_types_folder, "*.trbx")):
    turbine_type = wf.Scripting.TurbineType(trbxFile)

    if turbine_type.Name in existing_turbine_type_names:
        print(f"{turbine_type.Name} already loaded! Skipping!")
        continue

    wf.Workbook.TurbineTypes.Add(turbine_type)
    print( f"Loaded turbine type {turbine_type.Name} from file {trbxFile}")
    


### Define turbine locations
* parse locations from file into a dataframe
* Set initiation mast on each turbine manually

In [ ]:
turbine_name_prefix = "T"
turbine_count = 0


for wind_farm_name in windfarms_df.index:
    wind_farm_row = windfarms_df.loc[wind_farm_name]
    print(wind_farm_row)
    
    # add wind farm if it doesn't exist
    if wf.Workbook.WindFarms[wind_farm_name] == None:
        wf.Workbook.WindFarms.Add( wf.Scripting.WindFarm(wind_farm_name))
    
    wf.Workbook.WindFarms[wind_farm_name].IsNeighbour = bool(wind_farm_row["is_neighbor"])

    turbine_type = wf.Workbook.TurbineTypes[wind_farm_row["turbine_type_name"]]
    if turbine_type == None:
        turbine_type_name = wind_farm_row["turbine_type_name"]
        print(f"ERROR: Turbine type: {turbine_type_name }, requested doesn't exist in workbook! " )
    
    # add the turbines to the wind farm
    turbine_locations = wf.Toolbox.ImportLocationsFromShapeFile( wind_farm_row["shapefile_path"])
    for location in turbine_locations:
        turbine_count = turbine_count +1
        turbine_name = f"{turbine_name_prefix}{turbine_count}"
        wf.Workbook.WindFarms[wind_farm_name].Turbines.Add( 
            wf.Scripting.Turbine( 
                turbine_name, 
                wf.Scripting.Location(location.X, location.Y),
                turbine_type))
#wf.Scripting.IReadable2DLocation()

### Import flow model

In [ ]:
if (wf.Toolbox.IsWaspAvailable(wf.Scripting.WAsPVersion.Version12) == wf.Scripting.WAsPStatus.Available):
    print("\t running WAsP 12 flow model")
    wf.Workbook.ModelSettings.FlowSettings.FlowModelType = wf.Scripting.FlowModelType.WaspFromFreqDist
    wf.Workbook.ModelSettings.FlowSettings.WAsPParameters.WAsPVersion = wf.Scripting.WAsPVersion.Version11
else:
    print("\t running simple flow model")
    wf.Workbook.ModelSettings.FlowSettings.FlowModelType = wf.Scripting.FlowModelType.Simple

wf.Toolbox.CalculateWindFlow()
print("Completed wind flow calculation")

### Run energy calculation

In [ ]:
# Set some energy calculation settings
energySettings = wf.Workbook.ModelSettings.EnergySettings
energySettings.WakeModelType =  wf.Scripting.WakeModelType.EddyViscosity
energySettings.ApplyLargeWindFarmCorrection = True
energySettings.CalculationToUse = wf.Scripting.EnergyCalculationToUseType.New
energySettings.CalculateEfficiencies = True
energySettings.LargeWindFarmCorrectionSettings.BaseRoughness = 0.0002
energySettings.LargeWindFarmCorrectionSettings.IncreasedRoughness = 0.0192
energySettings.LargeWindFarmCorrectionSettings.DistanceInDiametersToStartOfRecovery = 120
energySettings.NumberOfDirectionSectors = 180


In [ ]:
# set some FPM export settings
energySettings.ExportFPMsAfterCalculation = True
energySettings.ExportFPMs.FolderPath = os.getcwd()

energySettings.ExportFPMs.AmbientTurbulenceAtTurbines_MastBinning.Export = True
energySettings.ExportFPMs.AmbientTurbulenceAtTurbines_MastBinning.ExportFilePrefix = "AmbientTurbulenceAtTurbines_"
energySettings.ExportFPMs.DesignTurbulenceByWindSpeed_TurbineBinning.Export  = False
energySettings.ExportFPMs.DesignTurbulenceByWindSpeedAndDirection_TurbineBinning.Export  = False
energySettings.ExportFPMs.MastToTurbineSpeedUps.Export  = False
energySettings.ExportFPMs.ProbabilityDistribution_MastBinning.Export  = False
energySettings.ExportFPMs.TerrainAngle.Export  = False
energySettings.ExportFPMs.TotalWindFarmElectricPower.Export  = False
energySettings.ExportFPMs.TurbineElectricPower_MastBinning.Export   = False
energySettings.ExportFPMs.TurbineHubHeightAmbientWindSpeed_MastBinning.Export  = False
energySettings.ExportFPMs.TurbineOperational_MastBinning.Export  = False
energySettings.ExportFPMs.WakedTurbulenceAtTurbines_MastBinning.Export  = False

wf.Toolbox.CalculateEnergy()

In [ ]:
# Compute the full yield
print (f'Computing energy')
results_scenario = wf.Toolbox.CalculateEnergy()
print ('Finished energy calculation!')


# Read some results
subject_wind_farm_names = windfarms_df[(windfarms_df["is_neighbor"]== False)].index
full_yeild = 0
for subject_farm_name in subject_wind_farm_names:
    subjectWindFarm = [x for x in results_scenario.WindFarms if x.Name in subject_wind_farm_names][0]
    farm_full_yeild = results_scenario.FarmTotalYields.GetVariantResult("Full").GetValueForFarm(subjectWindFarm).Value / 1e6	
    full_yeild = full_yeild + farm_full_yeild
    print( str.format("Full yield for wind farm {0}:\t{1:.2f} GWh/annum\n", subjectWindFarm.Name, farm_full_yeild))

print( str.format("Full yield for all subject wind farms = {0:.2f} GWh/annum\n", full_yeild))

In [ ]:
print(wf.Toolbox.get_CurrentWorkbookPath())
wf.Toolbox.Save()

### Power time series
To run the power time series calculation, you will need to do some setup that can't be configured by scripting yet.
* Open the workbook at the path above
* Go to Energy > Power time series
* Open "Select wind climate inputs" and choose the time series data to fit your initiation strategy for any wind climates listed that initiate at least 1 turbine.

Note, performance can be a problem for power time series. You may want to resample time series to hourly data to compute 6 times fewer records. 

The following settings can be defined in scripting, and you will see them in the GUI:

In [ ]:
wf.Workbook.ModelSettings.PowerTimeSeriesCalculationSettings.UseAirDensitySingleValue = True
wf.Workbook.ModelSettings.PowerTimeSeriesCalculationSettings.AirDensityMeanTimeSeries = None 
# air density settings are then used 

wf.Workbook.ModelSettings.PowerTimeSeriesCalculationSettings.UseTurbulenceIntensity = True # true means use the turbulence intensity distribution, but you have to define that in the GUI
wf.Workbook.ModelSettings.PowerTimeSeriesCalculationSettings.DirectionMeanTimeSeries = M3_wd94_Mean

# Exporting the power time series to file. If you don't do this, you can't access it later.
# Note there is currently a significant performance impact from exporting the time series to file. We have plans to optimise that 
# our focus when building this had been more on curtailment estimation with power time series
wf.Workbook.ModelSettings.PowerTimeSeriesCalculationSettings.ShouldExportIntermediateResults = True
wf.Workbook.ModelSettings.PowerTimeSeriesCalculationSettings.IntermediateResultsFolderPath = os.path.abspath(os.getcwd())

wf.Workbook.ModelSettings.PowerTimeSeriesCalculationSettings.ApplyCurtailmentRules = False 
# these rules can be defined on wind farms, turbines and turbine types, see CurtailmentRule - https://mysoftware.dnv.com/download/public/renewables/windfarmer/manuals/latest/api/Scripting.CurtailmentRule.html
# wf.Workbook[""].Turbines[""].CurtailmentRules.Add() 

In [ ]:
# You can run the power time series here if you have already defined the inputs in the GUI.
#wf.Toolbox.CalculatePowerTimeSeries()

In [ ]:
# calling new workbook closes the workbook previously being editted here.
wf.Toolbox.NewWorkbook()